## Packages

In [1]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima
from arch import arch_model
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

## Loading the data

In [ ]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", 
                              end = "2019-09-01", interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

[                       0%                       ]

In [ ]:
df_comp = raw_data.copy()

In [ ]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [ ]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

## Creating Returns

In [ ]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1).mul(100)
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1).mul(100)
df_comp['ret_dax'] = df_comp.dax.pct_change(1).mul(100)
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1).mul(100)

In [ ]:
df_comp['norm_ret_spx'] = df_comp.ret_spx.div(df_comp.ret_spx[1])*100
df_comp['norm_ret_ftse'] = df_comp.ret_ftse.div(df_comp.ret_ftse[1])*100
df_comp['norm_ret_dax'] = df_comp.ret_dax.div(df_comp.ret_dax[1])*100
df_comp['norm_ret_nikkei'] = df_comp.ret_nikkei.div(df_comp.ret_nikkei[1])*100

## Splitting the Data

In [ ]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

## Fitting a Model

In [ ]:
model_ar= ARIMA(df.ftse, order=(1,0,0))
results_ar = model_ar.fit()

## Simple Forecasting

In [ ]:
df.tail()

In [ ]:
# create variables that will help us change the periods easily instead of typing them up every time
# make sure the start and end dates are business days, otherwise the code will result in an error
start_date = "2014-07-15"
end_date = "2015-01-01"

In [ ]:
df_pred = results_ar.predict(start=start_date , end=end_date)

In [ ]:
df_pred[start_date:end_date].plot(figsize =(20,5), color = "red")
df_test.ftse[start_date:end_date].plot(color = "blue")
plt.title("Predictions VS Actual", size = 24)
plt.show()

In [ ]:
end_date1 = "2019-10-23"
df_pred = results_ar.predict(start=start_date , end=end_date1)

In [ ]:
df_pred[start_date:end_date1].plot(figsize =(20,5), color = "red")
df_test.ftse[start_date:end_date1].plot(color = "blue")
plt.title("Predictions VS Actual", size = 24)
plt.show()

### Using Returns Instead

In [ ]:
model_ret_ar= ARIMA(df.ret_ftse[1:], order=(1,0,0))
results_ret_ar = model_ret_ar.fit()

df_pred_ar = results_ret_ar.predict(start=start_date , end=end_date)
    
df_pred_ar[start_date:end_date].plot(figsize =(20,5), color = "red")
df_test.ret_ftse[start_date:end_date].plot(color = "blue")
plt.title("Predictions VS Actual (Returns)", size = 24)
plt.show()

In [ ]:
results_ret_ar.summary()

In [ ]:
end_date2 = "2014-08-18"
model_ret_ar= ARIMA(df.ret_ftse[1:], order=(5,0,0))
results_ret_ar = model_ret_ar.fit()

df_pred_ar = results_ret_ar.predict(start=start_date , end=end_date2)
    
df_pred_ar[start_date:end_date2].plot(figsize =(20,5), color = "red")
df_test.ret_ftse[start_date:end_date2].plot(color = "blue")
plt.title("Predictions VS Actual (Returns)", size = 24)
plt.show()

### MA

In [ ]:
model_ret_ma= ARIMA(df.ret_ftse[1:], order=(0,0,1))
results_ret_ma = model_ret_ma.fit()

df_pred_ma = results_ret_ma.predict(start=start_date , end=end_date)
    
df_pred_ma[start_date:end_date].plot(figsize =(20,5), color = "red")
df_test.ret_ftse[start_date:end_date].plot(color = "blue")
plt.title("Predictions VS Actual (Returns)", size = 24)
plt.show()

In [ ]:
df_pred_ma.head()

### ARMA

In [ ]:
model_ret_arma= ARIMA(df.ret_ftse[1:], order=(1,0,1))
results_ret_arma = model_ret_arma.fit()

df_pred_arma = results_ret_arma.predict(start=start_date , end=end_date)
    
df_pred_arma[start_date:end_date].plot(figsize =(20,5), color = "red")
df_test.ret_ftse[start_date:end_date].plot(color = "blue")
plt.title("Predictions VS Actual (Returns)", size = 24)
plt.show()

In [ ]:
df_pred_arma.head()

In [ ]:
df_pred_arma.tail()

### ARMAX

In [ ]:
model_ret_armax= ARIMA(df.ret_ftse[1:], exog=df[["ret_spx","ret_dax","ret_nikkei"]][1:], order=(1,0,1))
results_ret_armax = model_ret_armax.fit()

df_pred_armax = results_ret_armax.predict(start=start_date , end=end_date, exog=df_test[["ret_spx","ret_dax","ret_nikkei"]][start_date:end_date])
    
df_pred_armax[start_date:end_date].plot(figsize =(20,5), color = "red")
df_test.ret_ftse[start_date:end_date].plot(color = "blue")
plt.title("Predictions VS Actual (Returns)", size = 24)
plt.show()

### SARMA

In [ ]:
model_ret_sarma= SARIMAX(df.ret_ftse[1:], order=(3,0,3),seasonal_order=(3,0,2,5))
results_ret_sarma = model_ret_sarma.fit()

df_pred_sarma = results_ret_sarma.predict(start=start_date , end=end_date)
    
df_pred_sarma[start_date:end_date].plot(figsize =(20,5), color = "red")
df_test.ret_ftse[start_date:end_date].plot(color = "blue")
plt.title("Predictions VS Actual (SARMA)", size = 24)
plt.show()

### SARIMAX

In [ ]:
model_ret_sarimax= SARIMAX(df.ret_ftse[1:], exog=df[["ret_spx","ret_dax","ret_nikkei"]][1:], order=(3,0,4),seasonal_order=(3,0,2,5))
results_ret_sarimax = model_ret_sarimax.fit()

df_pred_sarimax = results_ret_sarimax.predict(start=start_date , end=end_date, exog=df_test[["ret_spx","ret_dax","ret_nikkei"]][start_date:end_date])
    
df_pred_sarimax[start_date:end_date].plot(figsize =(20,5), color = "red")
df_test.ret_ftse[start_date:end_date].plot(color = "blue")
plt.title("Predictions VS Actual (Returns)", size = 24)
plt.show()

### Auto ARIMA

In [ ]:
model_auto = auto_arima(df.ret_ftse[1:],exogenous=df[["ret_spx","ret_dax","ret_nikkei"]][1:], 
                        m = 5, max_p = 5, max_q=5,max_P = 5, max_Q=5)

In [ ]:
df_auto_pred = pd.DataFrame(model_auto.predict(n_periods=len(df_test[start_date:end_date])),
                            exogenous=df_test[["ret_spx","ret_dax","ret_nikkei"]][start_date:end_date]
                            ,index = df_test[start_date:end_date].index)


In [ ]:
    
df_auto_pred.plot(figsize =(20,5), color = "red")
df_test.ret_ftse[start_date:end_date].plot(color = "blue")
plt.title("Auto model Predictions VS Real Data", size = 24)
plt.show()

### Comparing All the Models (Returns)

### Forecasting Volatility

### Multvariate Regression Model